In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path = '/content/drive/My Drive/Colab Notebooks/3_Corpora/mrpc-sentiment-triplet-1056-source.csv'

In [ ]:
import pandas as pd

df = pd.read_csv(file_path)
df.head()

,sentence1,sentence2,label,idx,sentiment1_label,sentiment1_score,sentiment2_label,sentiment2_score
0,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",0,3,NEU,0.814170,POS,0.745833
1,"The Nasdaq had a weekly gain of 17.27 , or 1.2...",The tech-laced Nasdaq Composite .IXIC rallied ...,0,6,NEU,0.856691,POS,0.534851
2,"That compared with $ 35.18 million , or 24 cen...",Earnings were affected by a non-recurring $ 8 ...,0,8,NEU,0.973141,NEG,0.682705
3,"Shares of Genentech , a much larger company wi...",Shares of Xoma fell 16 percent in early trade ...,0,10,POS,0.834054,NEU,0.523223
4,"Gyorgy Heizler , head of the local disaster un...","The head of the local disaster unit , Gyorgy H...",0,15,NEU,0.968184,NEG,0.742662


In [ ]:
df['label'].value_counts()

1    530
0    526
Name: label, dtype: int64

In [ ]:
# Take only paraphrases
df = df[df['label'] == 1]

In [ ]:
import pandas as pd
import numpy as np

# Define mapping for sentiment labels to attribute names
sentiment_mapping = {
    'NEU': 'original',
    'NEG': 'negative',
    'POS': 'positive'
}

# Initialize new columns for sentences, labels, and scores for each sentiment category
for sentiment in ['original', 'positive', 'negative']:
    df[f'{sentiment}_sentence'] = None
    df[f'{sentiment}_sentiment_label'] = None
    df[f'{sentiment}_sentiment_score'] = None

# Function to assign sentences, labels, and scores based on sentiment
def assign_sentiment(row):
    for i in range(1, 3):
        sentiment_key = row[f'sentiment{i}_label']  # POS, NEG, NEU
        sentiment = sentiment_mapping.get(sentiment_key)  # Map to original, positive, negative
        if sentiment:  # If mapping exists
            row[f'{sentiment}_sentence'] = row[f'sentence{i}']
            row[f'{sentiment}_sentiment_label'] = sentiment_key
            row[f'{sentiment}_sentiment_score'] = row[f'sentiment{i}_score']
    return row

# Apply the function to each row
df = df.apply(assign_sentiment, axis=1)

df.drop(['sentence1', 'sentence2', 'sentiment1_label', 'sentiment1_score', 'sentiment2_label', 'sentiment2_score', 'idx'], axis=1, inplace=True)

# Assuming df is your DataFrame after sentiment analysis processing
desired_order = [
    'original_sentence', 'positive_sentence', 'negative_sentence',
    'original_sentiment_label', 'original_sentiment_score',
    'positive_sentiment_label', 'positive_sentiment_score',
    'negative_sentiment_label', 'negative_sentiment_score'
]

# Reorder the DataFrame columns
df = df[desired_order]

# Replace NaN in text columns with empty strings
for col in df.select_dtypes(include=['object']).columns:
    df[col].fillna('', inplace=True)

# Replace NaN, inf, and -inf in numeric columns with 0
for col in df.select_dtypes(include=[np.number]).columns:
    df[col].replace([np.nan, np.inf, -np.inf], 0, inplace=True)


df.head()

,original_sentence,positive_sentence,negative_sentence,original_sentiment_label,original_sentiment_score,positive_sentiment_label,positive_sentiment_score,negative_sentiment_label,negative_sentiment_score
0,"Around 0335 GMT , Tab shares were up 19 cents ...","Tab shares jumped 20 cents , or 4.6 % , to set...",,NEU,0.814170,POS,0.745833,,0.000000
1,"The Nasdaq had a weekly gain of 17.27 , or 1.2...",The tech-laced Nasdaq Composite .IXIC rallied ...,,NEU,0.856691,POS,0.534851,,0.000000
2,"That compared with $ 35.18 million , or 24 cen...",,Earnings were affected by a non-recurring $ 8 ...,NEU,0.973141,,0.000000,NEG,0.682705
3,Shares of Xoma fell 16 percent in early trade ...,"Shares of Genentech , a much larger company wi...",,NEU,0.523223,POS,0.834054,,0.000000
4,"Gyorgy Heizler , head of the local disaster un...",,"The head of the local disaster unit , Gyorgy H...",NEU,0.968184,,0.000000,NEG,0.742662


In [ ]:
import gspread
from google.oauth2.service_account import Credentials

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
creds = Credentials.from_service_account_file('/content/drive/My Drive/Colab Notebooks/cedar-card-413120-c97e74ee71de.json', scopes=scope)
client = gspread.authorize(creds)

In [ ]:
spreadsheet_url = 'https://docs.google.com/spreadsheets/d/1eX0atV1WvGkkOS-JMN5XH84IfiJoshalGmMhEwJfkY0'
sheet = client.open_by_url(spreadsheet_url).sheet1

In [ ]:
sheet.clear()
# Convert the DataFrame to a list of lists, where each sublist is a row
dataframe_values = df.values.tolist()
# Reinsert headers at the beginning
dataframe_values.insert(0, df.columns.to_list())

# Update the sheet with the DataFrame's values
sheet.update('A1', dataframe_values)

{'spreadsheetId': '1eX0atV1WvGkkOS-JMN5XH84IfiJoshalGmMhEwJfkY0',
 'updatedRange': 'Sheet1!A1:I1057',
 'updatedRows': 1057,
 'updatedColumns': 9,
 'updatedCells': 9513}